In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Myriad Pro'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600

import numpy as np
import scipy.stats as stats

from scipy.stats import kstest, cramervonmises
import tensorflow as tf
import tensorflow_probability as tfp
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import time

from ipcc_colormap import *
from utils import *

from matplotlib import gridspec
from matplotlib.colorbar import ColorbarBase

import pickle

coastline = gpd.read_file('/home/mizu_home/xp53/nas/home/coastlines-split-SGregion/lines.shp')
mask = np.loadtxt('mask.txt')

ipcc_blue = (112.0/255, 160.0/255, 205.0/255, 1.0)
ipcc_orange = (196.0/255, 121.0/255, 0.0/255, 1.0)

tmp_cmap = ipcc_cmap()
tmp_cmap.read_rgb_data_from_excel()
;

2024-09-04 22:17:25.251608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 22:17:25.317372: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


''

In [2]:
# DATA PREPARATION
# the station-based rainfall data is already organized in a N by P matrix
rain_obs = np.loadtxt('data/sta_monthly.csv')

# the simulated rainfall data is reshaped to a 2d matrix of size N by (W x L)
rain_sim_flatten = np.loadtxt('data/wrf_monthly.csv')
rain_sim = rain_sim_flatten.reshape(rain_sim_flatten.shape[0], 120, 160)

# sim_sel constains a P by 4 matrix
# the first two columns are the row and column indices of grids corresponding to the stations
# the last two columns are the lons and lats of grids corresponding to the stations
sim_sel = np.loadtxt('data/wrf_loc.csv')
sim_idx = sim_sel[:, :2].astype(int)

# select simulated rainfall at stations
wrf_sta = np.array([rain_sim[:, i, j] for (i, j) in sim_idx]).T

# lats and lons of all grids are stored in a 2d matrix of 2 by (W x L)
# the first row is lons and the second row is lats
longlat = np.loadtxt('data/lonlat.txt')
lons = longlat[0, :].reshape(120, 160)
lats = longlat[1, :].reshape(120, 160)

# read station lons (3rd column) and lats (4th column)
sta_loc = np.genfromtxt('data/sta_lookup_new.csv', delimiter=',')[:, 2:]


In [3]:
rain1 = np.zeros((40*12, 120, 160))
rain2 = np.zeros((40*12, 120, 160))
ru1 = np.zeros((12, 120, 160))
ru2 = np.zeros((12, 120, 160))
# interpolate to all grids
gpr_ = gp_interpolator(P = sim_idx.shape[0])

sg_plotter = sg_map_plotter(lons, lats, coastline)
cmap = tmp_cmap.get_ipcc_cmap('seq', 'prec', 20)

fig = plt.figure(figsize=(11, 10))
gs = gridspec.GridSpec(4, 3, height_ratios=[1,1,1,1], bottom=0.1, top=0.95, left=0.05, right=0.95,
                       wspace=0.05, hspace=0.05)  # Leave space at the bottom for the colorbar

axes = [plt.subplot(gs[i, j], projection=crs.PlateCarree()) for i in range(4) for j in range(3)]

for i, ax in enumerate(axes):
    t1 = time.time()
    gpr_.read_rainfall(rain_obs[i::12, :], wrf_sta[i::12, :])
    gpr_.sn_converge()
    r1, var1 = gpr_.predict(rain_sim_flatten[i::12, :])
    r1 = np.reshape(r1.T, (40, 120, 160))
    var1 = np.reshape(np.diag(var1), (120, 160))

    x1, _ = gpr_.predict(gpr_.wrf_sta)
    ex = rain_obs[i::12, :] - x1.T
    pause = 1

    r2 = np.zeros((40, 120, 160))
    var_y = np.zeros((120, 160))
    for k in range(40):
        tdata = np.concatenate((sta_loc, ex[k, :][:, None]), axis = 1)
        OK = OrdinaryKriging(
                tdata[:, 0], tdata[:, 1], tdata[:, 2],
                variogram_model = 'gaussian'
            )
        ey, tmp_var = OK.execute('points', lons.flatten(), lats.flatten())
        var_y += 1/40 * tmp_var.reshape(120, 160)
        r2[k, :, :] = r1[k, :, :] + ey.reshape(120, 160)
    r1[r1 < 0] = 0
    r2[r2 < 0] = 0

    rain1[i::12, :, :] = r1
    rain2[i::12, :, :] = r2
    ru1[i, :, :] = np.sqrt(var1)
    ru2[i, :, :] = np.sqrt(var1 + var_y)

    # sg_plotter.plot_(ax, np.mean(rain2[i::12, :, :], axis = 0), cmap = cmap, vmin = 0, vmax = 400)
    sg_plotter.plot_(ax, ru2[i, :, :], cmap = cmap, vmin = 0, vmax = 400)
    sg_plotter.plot_scatter(ax, sta_loc, size = 15)
    print('Time taken: ', time.time() - t1)

# Adjust the colorbar placement
cbar_ax = fig.add_axes([0.06, 0.07, 0.88, 0.015])  # Position: [left, bottom, width, height]
bounds = np.linspace(0, 400, 21)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.ax.tick_params(labelsize=12)
# make label font larger and bold
cb.set_label('Rainfall Climatology [mm]', fontsize=12, fontweight='bold')

fig.savefig('figures_two_stage/poster_var_his_twostage.pdf')


Time taken:  19.956090450286865
Time taken:  19.977900981903076
Time taken:  19.73865580558777
Time taken:  20.74614953994751
Time taken:  21.007766246795654
Time taken:  21.69706416130066
Time taken:  21.679450750350952
Time taken:  20.96605348587036
Time taken:  20.57366633415222
Time taken:  21.0658278465271
Time taken:  21.149197101593018
Time taken:  21.007020235061646


In [4]:
fig = plt.figure(figsize=(11, 10))
gs = gridspec.GridSpec(4, 3, height_ratios=[1,1,1,1], bottom=0.1, top=0.95, left=0.05, right=0.95,
                       wspace=0.05, hspace=0.05)  # Leave space at the bottom for the colorbar

axes = [plt.subplot(gs[i, j], projection=crs.PlateCarree()) for i in range(4) for j in range(3)]

for i, ax in enumerate(axes):
    # sg_plotter.plot_(ax, np.mean(rain2[i::12, :, :], axis = 0), cmap = cmap, vmin = 0, vmax = 400)
    sg_plotter.plot_(ax, ru1[i, :, :], cmap = cmap, vmin = 0, vmax = 400)
    sg_plotter.plot_scatter(ax, sta_loc, size = 15)
    print('Time taken: ', time.time() - t1)

# Adjust the colorbar placement
cbar_ax = fig.add_axes([0.06, 0.07, 0.88, 0.015])  # Position: [left, bottom, width, height]
bounds = np.linspace(0, 400, 21)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.ax.tick_params(labelsize=12)
# make label font larger and bold
cb.set_label('Rainfall Climatology [mm]', fontsize=12, fontweight='bold')

fig.savefig('figures_two_stage/poster_var_his.pdf')

Time taken:  110.28209257125854
Time taken:  110.88883018493652
Time taken:  111.698655128479
Time taken:  112.4423258304596
Time taken:  113.31625747680664
Time taken:  114.33006715774536
Time taken:  115.46774625778198
Time taken:  116.71706652641296
Time taken:  118.09439873695374
Time taken:  119.58853888511658
Time taken:  121.17671751976013
Time taken:  122.86493277549744
